### concatenate the features in the same string or not?  
chatgpt  
"If features 3, 4, and 5 are extracted from feature 1, it indicates a dependency or relationship between these features. In this case, it would be more appropriate to choose combine all the features into a single list and use a single TfidfVectorizer to transform them. By treating all the features together as a collective input, you can capture the interdependencies and relationships between them, allowing the vectorizer to consider the combined textual information from feature 1, feature 3, feature 4, and feature 5. This approach will help to capture the overall textual patterns and relationships within the data."  

"Concatenating Features: When you concatenate all the features into the same feature, you create a single unified representation that captures the information from all the features. This approach can be useful when there is a strong interdependence or correlation between the different features, and combining them into a single feature provides a more comprehensive representation of the data. By doing so, you create a higher-dimensional feature space that can potentially capture complex relationships among the features. However, this approach may also introduce noise or irrelevant information if some of the individual features are not informative for the task at hand."  

"Combining all the features might lead to the loss of feature-specific information. For example, if you have separate NER (Named Entity Recognition) tags or keywords, they might lose their distinctiveness when combined with other textual content. Scale sensitivity: If the scale or magnitude of one feature dominates the others, it might overshadow the contributions of other features during the vectorization process."  

"Each feature can be vectorized independently, allowing the vectorizer to capture specific characteristics of each feature separately."

In [1]:
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from scipy.sparse import hstack
import numpy as np
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize
from scipy.sparse import hstack

In [2]:
import numpy as np
np.random.seed(42)  # Set random seed for numpy

import random
random.seed(42)  # Set random seed for random module

In [3]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")
df.head()

,date,title,reason,description,zone,narrative_tfidf,narrative_embeddings,narrative_bert,narrative_tfidf_title,narrative_bert_title,...,embeddings_title,tfidf_keywords,embeddings_keywords,events_clean,results_final,events_tfidf,events_embeddings,orgs,locs,entities
0,2021-10-11 12:26:32,CTT - Encomenda entregue danificada e com etiq...,0,Bom dia venho por este meio apresentar uma rec...,Lisboa,bom dia venho meio apresentar reclamação ctt e...,bom dia venho por este meio apresentar uma rec...,Bom dia venho por este meio apresentar uma rec...,ctt encomenda entregar danificar etiqueta envi...,ctt - encomenda entregue danificada e com etiq...,...,ctt - encomenda entregue danificada e com etiq...,apresentar reclamação caixa figura envio israe...,apresentar uma reclamação tinha caixa figura e...,venho apresentar enviei destinado feito proteg...,"[('CTT', 'B-ORGANIZACAO'), ('Israel', 'B-LOCAL...",venho apresentar enviar destinar fazer protege...,venho apresentar enviei destinado feito proteg...,"CTT, Aboboda",Israel,"CTT, Aboboda, Israel"
1,2021-08-31 23:20:15,CTT - Carta com certificado rasgado!,1,- Boa Noite!\r\n\r\n- Venho por este meio recl...,Coimbra,bom noite venho meio reclamar data ás durante ...,- boa noite!\r\n\r\n- venho por este meio recl...,- Boa Noite! Venho por este meio reclamar que ...,ctt carta certificar rasgar bom noite venho me...,ctt - carta com certificado rasgado! - Boa No...,...,ctt - carta com certificado rasgado !,lilia pimentel certificar profissional passage...,lilia pimentel certificado profissional passag...,venho reclamar deparei -me continha tem,"[('Lilia', 'B-PESSOA'), ('Pimentel', 'I-PESSOA')]",venho reclamar deparar conter,venho reclamar deparei - me continha tem,NaN,NaN,NaN
2,2021-02-01 19:08:32,CTT - Encomenda registada que nunca chegou ao ...,2,"Boa tarde,\r\nNo dia 17 de Dezembro de 2020 en...",Aveiro,bom tarde dia dezembro enviar encomenda França...,"boa tarde,\r\nno dia 00 de dezembro de 0000 en...","Boa tarde, No dia 17 de Dezembro de 2020 envie...",ctt encomenda registar nunca chegar destine o ...,ctt - encomenda registada que nunca chegou ao ...,...,ctt - encomenda registada que nunca chegou ao ...,frança correio registar frança correio encomen...,frança em correio registado frança em correio ...,enviei registado vejo chegou saiu chegou tem p...,"[('França', 'B-LOCAL'), ('Roissy', 'B-ORGANIZA...",enviar registar vejo chegar sair chegar passar...,enviei registado vejo chegou saiu chegou tem p...,Roissy,"França, ROISSY","Roissy, França, ROISSY"
3,2018-04-19 09:04:34,CTT - Correio Verde,2,"Bom dia,\n\nPretendia esclarecer uma situação ...",Lisboa,bom dia pretender esclarecer situação ocorrer ...,"bom dia,\n\npretendia esclarecer uma situação ...","Bom dia, Pretendia esclarecer uma situação que...",ctt correio verde bom dia pretender esclarecer...,"ctt - correio verde Bom dia, Pretendia esclar...",...,ctt - correio verde,pretender esclarecer situação pretender esclar...,pretendia esclarecer uma situação pretendia es...,pretendia esclarecer ocorreu desloquei dado fe...,"[('El', 'B-LOCAL'), ('Corte', 'I-LOCAL'), ('In...",pretender esclarecer ocorrer desloquei dar fec...,pretendia esclarecer ocorreu desloquei dado fe...,Correio,"El Corte Inglés, Lisboa Verde","Correio, El Corte Inglés, Lisboa Verde"
4,2019-07-04 10:20:41,"CTT - Aviso de recepção da carta registada, pr...",2,venho por este meio solicitar a vossa ajuda pa...,Porto,venho meio solicitar vosso ajudar saber onde e...,venho por este meio solicitar a vossa ajuda pa...,venho por este meio solicitar a vossa ajuda pa...,ctt aviso recepção carta registar preciso urge...,"ctt - aviso de recepção da carta registada, pr...",...,"ctt - aviso de recepção da carta registada , p...",papal centro pois preciso urgentemente pois pr...,papais para o centro pois preciso dele urgente...,venho solicitar saber onde se encontra regista...,[],venho solicitar saber onde encontrar registar ...,venho solicitar saber onde se encontra regista...,NaN,NaN,NaN


In [4]:
df.columns

Index(['date', 'title', 'reason', 'description', 'zone', 'narrative_tfidf',
       'narrative_embeddings', 'narrative_bert', 'narrative_tfidf_title',
       'narrative_bert_title', 'narrative_embeddings_title', 'tfidf_title',
       'title_tfidf_nolemma', 'events', 'keywords', 'embeddings_title',
       'tfidf_keywords', 'embeddings_keywords', 'events_clean',
       'results_final', 'events_tfidf', 'events_embeddings', 'orgs', 'locs',
       'entities'],
      dtype='object')

In [74]:
print(df['entities'].isna().sum())
print(df['narrative_tfidf'].isna().sum())
print(df['tfidf_title'].isna().sum())
print(df['tfidf_keywords'].isna().sum())
print(df['events_tfidf'].isna().sum())

5091
0
4709
0
23


In [4]:
def preprocess_entities(row):
    if pd.isna(row)==False:
        entities = row.lower()  # Convert to lowercase
        entities = entities.replace(',', '')  # Remove commas
        entities = ' '.join(set(entities.split()))  # Convert to set to get unique values, then join back as a string
        return entities
df['entities'] = df['entities'].apply(preprocess_entities)

In [5]:
df['entities']

0                          ctt israel aboboda
1                                        None
2                               frança roissy
3        el corte verde inglés correio lisboa
4                                        None
                         ...                 
22425                portugal ctt reino unido
22426                                    None
22427                              aliexpress
22428                                  lisboa
22429                                    None
Name: entities, Length: 22430, dtype: object

# TF-IDF 

In [47]:
y=df['reason']
X = df[['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities']]

In [48]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

# Print the shape of each set
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (15701, 5)
y_train shape: (15701,)
X_val shape: (3364, 5)
y_val shape: (3364,)
X_test shape: (3365, 5)
y_test shape: (3365,)


In [79]:
def unique_words(data):
    # Split the string into individual words
    keywords_clean = data.fillna('')
    keywords_words = ' '.join(keywords_clean).split()
    # Get unique words
    unique_words = list(set(keywords_words))
    print(len(unique_words))

In [80]:
unique_words(df.narrative_tfidf)
unique_words(df.tfidf_keywords)
unique_words(df.events_tfidf)
unique_words(df.entities)
unique_words(df.tfidf_title)

25140
11339
9315
4617
2827


Generate the power set of features and evaluate different combinations to identify the most informative subset of features. 
Optimal feature combination is useful when you want to determine the combination of features that provides the best classification accuracy or predictive power for your specific task. This step will allow you to determine if incorporating additional features improves the classification performance beyond the baseline model.

In [49]:
features = ['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities']
feature_combinations = []

# Generate the power set of features
for r in range(1, len(features) + 1):
    combinations = itertools.combinations(features, r)
    feature_combinations.extend(combinations)

# Convert each feature combination to a list
feature_combinations = [list(combination) for combination in feature_combinations]
print(len(feature_combinations))

31


In [345]:
# rows where at least one of the columns has nan value
for t in feature_combinations:
    print(t, df[t].isna().any(axis=1).sum())

['narrative_tfidf'] 0
['tfidf_title'] 4709
['tfidf_keywords'] 0
['events_tfidf'] 23
['entities'] 5091
['narrative_tfidf', 'tfidf_title'] 4709
['narrative_tfidf', 'tfidf_keywords'] 0
['narrative_tfidf', 'events_tfidf'] 23
['narrative_tfidf', 'entities'] 5091
['tfidf_title', 'tfidf_keywords'] 4709
['tfidf_title', 'events_tfidf'] 4728
['tfidf_title', 'entities'] 8649
['tfidf_keywords', 'events_tfidf'] 23
['tfidf_keywords', 'entities'] 5091
['events_tfidf', 'entities'] 5109
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords'] 4709
['narrative_tfidf', 'tfidf_title', 'events_tfidf'] 4728
['narrative_tfidf', 'tfidf_title', 'entities'] 8649
['narrative_tfidf', 'tfidf_keywords', 'events_tfidf'] 23
['narrative_tfidf', 'tfidf_keywords', 'entities'] 5091
['narrative_tfidf', 'events_tfidf', 'entities'] 5109
['tfidf_title', 'tfidf_keywords', 'events_tfidf'] 4728
['tfidf_title', 'tfidf_keywords', 'entities'] 8649
['tfidf_title', 'events_tfidf', 'entities'] 8664
['tfidf_keywords', 'events_tfidf', 'ent

In [346]:
# rows where all the columns are NaN values
for t in feature_combinations:
    print(t,df[t].isna().all(axis=1).sum())

['narrative_tfidf'] 0
['tfidf_title'] 4709
['tfidf_keywords'] 0
['events_tfidf'] 23
['entities'] 5091
['narrative_tfidf', 'tfidf_title'] 0
['narrative_tfidf', 'tfidf_keywords'] 0
['narrative_tfidf', 'events_tfidf'] 0
['narrative_tfidf', 'entities'] 0
['tfidf_title', 'tfidf_keywords'] 0
['tfidf_title', 'events_tfidf'] 4
['tfidf_title', 'entities'] 1151
['tfidf_keywords', 'events_tfidf'] 0
['tfidf_keywords', 'entities'] 0
['events_tfidf', 'entities'] 5
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords'] 0
['narrative_tfidf', 'tfidf_title', 'events_tfidf'] 0
['narrative_tfidf', 'tfidf_title', 'entities'] 0
['narrative_tfidf', 'tfidf_keywords', 'events_tfidf'] 0
['narrative_tfidf', 'tfidf_keywords', 'entities'] 0
['narrative_tfidf', 'events_tfidf', 'entities'] 0
['tfidf_title', 'tfidf_keywords', 'events_tfidf'] 0
['tfidf_title', 'tfidf_keywords', 'entities'] 0
['tfidf_title', 'events_tfidf', 'entities'] 1
['tfidf_keywords', 'events_tfidf', 'entities'] 0
['narrative_tfidf', 'tfidf_title', 

In [50]:
new_feature_combinations = []

for t in feature_combinations:
    if not df[t].isna().all(axis=1).any():
        new_feature_combinations.append(t)

feature_combinations = new_feature_combinations

In [51]:
len(feature_combinations)

24

In [52]:
# rows where all the columns are NaN values
for t in feature_combinations:
    print(t,df[t].isna().all(axis=1).sum())

['narrative_tfidf'] 0
['tfidf_keywords'] 0
['narrative_tfidf', 'tfidf_title'] 0
['narrative_tfidf', 'tfidf_keywords'] 0
['narrative_tfidf', 'events_tfidf'] 0
['narrative_tfidf', 'entities'] 0
['tfidf_title', 'tfidf_keywords'] 0
['tfidf_keywords', 'events_tfidf'] 0
['tfidf_keywords', 'entities'] 0
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords'] 0
['narrative_tfidf', 'tfidf_title', 'events_tfidf'] 0
['narrative_tfidf', 'tfidf_title', 'entities'] 0
['narrative_tfidf', 'tfidf_keywords', 'events_tfidf'] 0
['narrative_tfidf', 'tfidf_keywords', 'entities'] 0
['narrative_tfidf', 'events_tfidf', 'entities'] 0
['tfidf_title', 'tfidf_keywords', 'events_tfidf'] 0
['tfidf_title', 'tfidf_keywords', 'entities'] 0
['tfidf_keywords', 'events_tfidf', 'entities'] 0
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf'] 0
['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'entities'] 0
['narrative_tfidf', 'tfidf_title', 'events_tfidf', 'entities'] 0
['narrative_tfidf', 'tfidf_keywords

### combine everything in a string and fillana('')
use all the features in a string and use combinations where at least one of the features exists for that row, 
- eg.x=['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities'] if a row doesnt have a value for column events_tfidf and entities, for feature_combination x we only use 'narrative_tfidf', 'tfidf_title', 'tfidf_keywords'. If a row has values for all columns in x we use 'narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities'.
- ['tfidf_title', 'entities'] has 1151 rows with no values for either column, so I didnt use this feature combination.
- is this correct?

In [351]:
algorithms = {'Linear SVC': LinearSVC(), 'XGBoost': XGBClassifier(), 'Random Forest': RandomForestClassifier(), 'KNN': KNeighborsClassifier()}
# Initialize the results table
cont=0

# Initialize an empty list to store individual result DataFrames
result_dfs = []
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    cont += 1
    X_train_combined = X_train[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    X_val_combined = X_val[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    
    # Transform the features using TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.95, min_df=2)
    X_train_combined_transformed = vectorizer.fit_transform(X_train_combined)
    X_val_combined_transformed = vectorizer.transform(X_val_combined)

    # Train and evaluate models for each algorithm
    for algorithm_name, algorithm in algorithms.items():
        # Train the model
        algorithm.fit(X_train_combined_transformed, y_train)

        # Predict labels
        y_pred = algorithm.predict(X_val_combined_transformed)

        # Calculate performance metrics
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred, average='macro')
        recall = recall_score(y_val, y_pred, average='macro')
        f1 = f1_score(y_val, y_pred, average='macro')

        # Create a DataFrame for the current combination and algorithm
        result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                                  'Algorithm': [algorithm_name],
                                  'Accuracy': [accuracy],
                                  'Precision': [precision],
                                  'Recall': [recall],
                                  'F1-Score': [f1]})
        # Append the DataFrame to the list
        result_dfs.append(result_df)
        
    print("Tested combination {} of 24".format(cont))

# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)

Tested combination 1 of 24
Tested combination 2 of 24
Tested combination 3 of 24
Tested combination 4 of 24
Tested combination 5 of 24
Tested combination 6 of 24
Tested combination 7 of 24
Tested combination 8 of 24
Tested combination 9 of 24
Tested combination 10 of 24
Tested combination 11 of 24
Tested combination 12 of 24
Tested combination 13 of 24
Tested combination 14 of 24
Tested combination 15 of 24
Tested combination 16 of 24
Tested combination 17 of 24
Tested combination 18 of 24
Tested combination 19 of 24
Tested combination 20 of 24
Tested combination 21 of 24
Tested combination 22 of 24
Tested combination 23 of 24
Tested combination 24 of 24


In [355]:
results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_tfidf,Linear SVC,0.657848,0.536161,0.451861,0.463292
1,narrative_tfidf,XGBoost,0.645957,0.518850,0.430046,0.435547
2,narrative_tfidf,Random Forest,0.647146,0.723007,0.384773,0.361212
3,narrative_tfidf,KNN,0.585612,0.441410,0.393639,0.399504
4,tfidf_keywords,Linear SVC,0.550832,0.417054,0.367611,0.373840
...,...,...,...,...,...,...
91,"tfidf_title, tfidf_keywords, events_tfidf, ent...",KNN,0.532996,0.385789,0.344612,0.345333
92,"narrative_tfidf, tfidf_title, tfidf_keywords, ...",Linear SVC,0.657848,0.514693,0.450477,0.458610
93,"narrative_tfidf, tfidf_title, tfidf_keywords, ...",XGBoost,0.659037,0.558996,0.454775,0.466421
94,"narrative_tfidf, tfidf_title, tfidf_keywords, ...",Random Forest,0.653389,0.642668,0.390058,0.367235


In [358]:
grouped_results = results.groupby('Algorithm')
algorithm_results = {}

for algorithm, group in grouped_results:
    algorithm_results[algorithm] = group.copy()

# Access the results for a specific algorithm
linear_svc_results = algorithm_results['Linear SVC']
xgboost_results = algorithm_results['XGBoost']
random_forest_results = algorithm_results['Random Forest']
knn_results = algorithm_results['KNN']

In [362]:
linear_svc_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_tfidf,Linear SVC,0.657848,0.536161,0.451861,0.463292
4,tfidf_keywords,Linear SVC,0.550832,0.417054,0.367611,0.373840
8,"narrative_tfidf, tfidf_title",Linear SVC,0.666766,0.545398,0.467301,0.479607
12,"narrative_tfidf, tfidf_keywords",Linear SVC,0.645660,0.504713,0.439868,0.448324
16,"narrative_tfidf, events_tfidf",Linear SVC,0.654281,0.523040,0.444775,0.453438
20,"narrative_tfidf, entities",Linear SVC,0.656361,0.521658,0.445430,0.453505
24,"tfidf_title, tfidf_keywords",Linear SVC,0.589477,0.468070,0.406095,0.416803
28,"tfidf_keywords, events_tfidf",Linear SVC,0.612961,0.458661,0.403819,0.408048
32,"tfidf_keywords, entities",Linear SVC,0.553805,0.413134,0.364185,0.368244
36,"narrative_tfidf, tfidf_title, tfidf_keywords",Linear SVC,0.650713,0.507202,0.444613,0.452760


In [363]:
xgboost_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
1,narrative_tfidf,XGBoost,0.645957,0.518850,0.430046,0.435547
5,tfidf_keywords,XGBoost,0.567479,0.499054,0.347494,0.339851
9,"narrative_tfidf, tfidf_title",XGBoost,0.659334,0.546550,0.451500,0.462223
13,"narrative_tfidf, tfidf_keywords",XGBoost,0.653983,0.513992,0.431460,0.436230
17,"narrative_tfidf, events_tfidf",XGBoost,0.646254,0.509562,0.431369,0.437710
21,"narrative_tfidf, entities",XGBoost,0.648335,0.513751,0.430992,0.436567
25,"tfidf_title, tfidf_keywords",XGBoost,0.612663,0.495460,0.402139,0.405232
29,"tfidf_keywords, events_tfidf",XGBoost,0.617122,0.506333,0.397368,0.398992
33,"tfidf_keywords, entities",XGBoost,0.571641,0.452767,0.347335,0.335146
37,"narrative_tfidf, tfidf_title, tfidf_keywords",XGBoost,0.664090,0.559104,0.456817,0.467326


In [364]:
random_forest_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
2,narrative_tfidf,Random Forest,0.647146,0.723007,0.384773,0.361212
6,tfidf_keywords,Random Forest,0.587396,0.601674,0.349572,0.331592
10,"narrative_tfidf, tfidf_title",Random Forest,0.656659,0.574844,0.390768,0.366980
14,"narrative_tfidf, tfidf_keywords",Random Forest,0.646849,0.751268,0.384187,0.360752
18,"narrative_tfidf, events_tfidf",Random Forest,0.642687,0.721068,0.381742,0.358471
22,"narrative_tfidf, entities",Random Forest,0.643282,0.734007,0.382268,0.358929
26,"tfidf_title, tfidf_keywords",Random Forest,0.615042,0.584783,0.372923,0.358382
30,"tfidf_keywords, events_tfidf",Random Forest,0.623662,0.731461,0.370756,0.350319
34,"tfidf_keywords, entities",Random Forest,0.585910,0.635910,0.349762,0.331687
38,"narrative_tfidf, tfidf_title, tfidf_keywords",Random Forest,0.652497,0.717148,0.388158,0.363408


In [365]:
knn_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
3,narrative_tfidf,KNN,0.585612,0.441410,0.393639,0.399504
7,tfidf_keywords,KNN,0.503567,0.365545,0.323084,0.320043
11,"narrative_tfidf, tfidf_title",KNN,0.587990,0.450141,0.404912,0.413692
15,"narrative_tfidf, tfidf_keywords",KNN,0.535375,0.384722,0.348183,0.349323
19,"narrative_tfidf, events_tfidf",KNN,0.565101,0.419330,0.377372,0.381407
23,"narrative_tfidf, entities",KNN,0.574316,0.409098,0.376383,0.377357
27,"tfidf_title, tfidf_keywords",KNN,0.517539,0.370369,0.332675,0.330430
31,"tfidf_keywords, events_tfidf",KNN,0.525268,0.381929,0.342256,0.343563
35,"tfidf_keywords, entities",KNN,0.497027,0.358182,0.320240,0.317553
39,"narrative_tfidf, tfidf_title, tfidf_keywords",KNN,0.548157,0.406624,0.363021,0.367616


In [12]:
from sklearn.svm import SVC
# Initialize the results table
cont=0

# Initialize an empty list to store individual result DataFrames
result_dfs = []
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    cont += 1
    X_train_combined = X_train[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    X_val_combined = X_val[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    
    # Transform the features using TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.95, min_df=2)
    X_train_combined_transformed = vectorizer.fit_transform(X_train_combined)
    X_val_combined_transformed = vectorizer.transform(X_val_combined)

    # Create an SVM classifier with the RBF kernel
    algorithm = SVC(kernel='rbf')
    # Fit the algorithm on the training data
    algorithm.fit(X_train_combined_transformed, y_train)
    # Predict labels
    y_pred = algorithm.predict(X_val_combined_transformed)

    # Calculate performance metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, average='macro')
    recall = recall_score(y_val, y_pred, average='macro')
    f1 = f1_score(y_val, y_pred, average='macro')

    # Create a DataFrame for the current combination and algorithm
    result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                              'Algorithm': ['SVC'],
                              'Accuracy': [accuracy],
                              'Precision': [precision],
                              'Recall': [recall],
                              'F1-Score': [f1]})
    # Append the DataFrame to the list
    result_dfs.append(result_df)
        
    print("Tested combination {} of 24".format(cont))

# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)

Tested combination 1 of 24
Tested combination 2 of 24
Tested combination 3 of 24
Tested combination 4 of 24
Tested combination 5 of 24
Tested combination 6 of 24
Tested combination 7 of 24
Tested combination 8 of 24
Tested combination 9 of 24
Tested combination 10 of 24
Tested combination 11 of 24
Tested combination 12 of 24
Tested combination 13 of 24
Tested combination 14 of 24
Tested combination 15 of 24
Tested combination 16 of 24
Tested combination 17 of 24
Tested combination 18 of 24
Tested combination 19 of 24
Tested combination 20 of 24
Tested combination 21 of 24
Tested combination 22 of 24
Tested combination 23 of 24
Tested combination 24 of 24


In [13]:
results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_tfidf,SVC,0.665874,0.578536,0.400938,0.374647
1,tfidf_keywords,SVC,0.585612,0.653618,0.347468,0.326642
2,"narrative_tfidf, tfidf_title",SVC,0.673900,0.629854,0.414884,0.396295
3,"narrative_tfidf, tfidf_keywords",SVC,0.657848,0.605538,0.396232,0.370747
4,"narrative_tfidf, events_tfidf",SVC,0.664388,0.660548,0.402568,0.379442
5,"narrative_tfidf, entities",SVC,0.666468,0.589695,0.400979,0.374748
6,"tfidf_title, tfidf_keywords",SVC,0.615339,0.649508,0.374526,0.358594
7,"tfidf_keywords, events_tfidf",SVC,0.636445,0.594367,0.380577,0.357013
8,"tfidf_keywords, entities",SVC,0.582937,0.660226,0.346827,0.326869
9,"narrative_tfidf, tfidf_title, tfidf_keywords",SVC,0.669144,0.656443,0.408699,0.388038


* **is it overfitting?**

In [54]:
algorithms = {'Linear SVC': LinearSVC(), 'XGBoost': XGBClassifier(), 'Random Forest': RandomForestClassifier()}
# Initialize the results table
cont=0

# Initialize an empty list to store individual result DataFrames
result_dfs = []
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Train Accuracy', 
                                'Train F1-Score', 'Test Accuracy', 'Test F1-Score'])

# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    cont += 1
    X_train_combined = X_train[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    X_val_combined = X_val[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    X_test_combined = X_test[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)

    # Transform the features using TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.95, min_df=2)
    X_train_combined_transformed = vectorizer.fit_transform(X_train_combined)
    X_val_combined_transformed = vectorizer.transform(X_val_combined)
    X_test_combined_transformed = vectorizer.transform(X_test_combined)

    # Train and evaluate models for each algorithm
    for algorithm_name, algorithm in algorithms.items():
        # Train the model
        algorithm.fit(X_train_combined_transformed, y_train)
        
        # Make predictions on the training set
        y_train_pred = algorithm.predict(X_train_combined_transformed)
        # Make predictions on the validation set
        y_val_pred = algorithm.predict(X_val_combined_transformed)
        
        # Calculate performance metrics on the training set
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_f1 = f1_score(y_train, y_train_pred, average='macro')

        # Calculate performance metrics on the validation set
        val_accuracy = accuracy_score(y_val, y_val_pred)
        val_f1 = f1_score(y_val, y_val_pred, average='macro')

        # Create a DataFrame for the current combination and algorithm
        result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                                  'Algorithm': [algorithm_name],
                                  'Train Accuracy': [train_accuracy], 
                                  'Train F1-Score':[train_f1], 
                                  'Val Accuracy':[val_accuracy], 
                                  'Val F1-Score':[val_f1]})
        # Append the DataFrame to the list
        result_dfs.append(result_df)
        
    print("Tested combination {} of {}".format(cont, len(feature_combinations)))

# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)

Tested combination 1 of 24
Tested combination 2 of 24
Tested combination 3 of 24
Tested combination 4 of 24
Tested combination 5 of 24
Tested combination 6 of 24
Tested combination 7 of 24
Tested combination 8 of 24
Tested combination 9 of 24
Tested combination 10 of 24
Tested combination 11 of 24
Tested combination 12 of 24
Tested combination 13 of 24
Tested combination 14 of 24
Tested combination 15 of 24
Tested combination 16 of 24
Tested combination 17 of 24
Tested combination 18 of 24
Tested combination 19 of 24
Tested combination 20 of 24
Tested combination 21 of 24
Tested combination 22 of 24
Tested combination 23 of 24
Tested combination 24 of 24


In [55]:
grouped_results = results.groupby('Algorithm')
algorithm_results = {}

for algorithm, group in grouped_results:
    algorithm_results[algorithm] = group.copy()

# Access the results for a specific algorithm
linear_svc_results = algorithm_results['Linear SVC']
xgboost_results = algorithm_results['XGBoost']
random_forest_results = algorithm_results['Random Forest']

In [56]:
linear_svc_results

,Features Combination,Algorithm,Train Accuracy,Train F1-Score,Val Accuracy,Val F1-Score
0,narrative_tfidf,Linear SVC,0.996815,0.996675,0.657848,0.463292
3,tfidf_keywords,Linear SVC,0.966817,0.965798,0.550832,0.373840
6,"narrative_tfidf, tfidf_title",Linear SVC,0.996688,0.996693,0.666766,0.479607
9,"narrative_tfidf, tfidf_keywords",Linear SVC,0.996306,0.996420,0.645660,0.448324
12,"narrative_tfidf, events_tfidf",Linear SVC,0.996752,0.996461,0.654281,0.453438
15,"narrative_tfidf, entities",Linear SVC,0.996815,0.996653,0.657253,0.460619
18,"tfidf_title, tfidf_keywords",Linear SVC,0.975734,0.976018,0.589477,0.416803
21,"tfidf_keywords, events_tfidf",Linear SVC,0.993631,0.993883,0.612961,0.408048
24,"tfidf_keywords, entities",Linear SVC,0.972295,0.971561,0.554697,0.368193
27,"narrative_tfidf, tfidf_title, tfidf_keywords",Linear SVC,0.996561,0.996825,0.650713,0.452760


In [57]:
xgboost_results

,Features Combination,Algorithm,Train Accuracy,Train F1-Score,Val Accuracy,Val F1-Score
1,narrative_tfidf,XGBoost,0.872938,0.840500,0.645957,0.435547
4,tfidf_keywords,XGBoost,0.720591,0.578032,0.567479,0.339851
7,"narrative_tfidf, tfidf_title",XGBoost,0.884339,0.855326,0.659334,0.462223
10,"narrative_tfidf, tfidf_keywords",XGBoost,0.878224,0.847486,0.653983,0.436230
13,"narrative_tfidf, events_tfidf",XGBoost,0.874785,0.844942,0.646254,0.437710
16,"narrative_tfidf, entities",XGBoost,0.880453,0.845950,0.653389,0.433922
19,"tfidf_title, tfidf_keywords",XGBoost,0.762308,0.649410,0.612663,0.405232
22,"tfidf_keywords, events_tfidf",XGBoost,0.813069,0.743216,0.617122,0.398992
25,"tfidf_keywords, entities",XGBoost,0.731164,0.598636,0.574019,0.342536
28,"narrative_tfidf, tfidf_title, tfidf_keywords",XGBoost,0.884530,0.854947,0.664090,0.467326


In [58]:
random_forest_results

,Features Combination,Algorithm,Train Accuracy,Train F1-Score,Val Accuracy,Val F1-Score
2,narrative_tfidf,Random Forest,0.999363,0.999328,0.645660,0.360305
5,tfidf_keywords,Random Forest,0.998790,0.998659,0.588288,0.332172
8,"narrative_tfidf, tfidf_title",Random Forest,0.999745,0.999685,0.655767,0.366625
11,"narrative_tfidf, tfidf_keywords",Random Forest,0.999363,0.999328,0.646552,0.360459
14,"narrative_tfidf, events_tfidf",Random Forest,0.999363,0.999328,0.641201,0.357612
17,"narrative_tfidf, entities",Random Forest,0.999363,0.999328,0.642985,0.358242
20,"tfidf_title, tfidf_keywords",Random Forest,0.999363,0.999334,0.615339,0.360449
23,"tfidf_keywords, events_tfidf",Random Forest,0.999236,0.999250,0.621879,0.349234
26,"tfidf_keywords, entities",Random Forest,0.998854,0.998792,0.587693,0.330070
29,"narrative_tfidf, tfidf_title, tfidf_keywords",Random Forest,0.999745,0.999685,0.654578,0.364428


### different vectorizers,  replace missing values with NA
- use all feature_combinations possible and replace NaN values with a special token like "NA" or "Unknown", instead of dropping the rows with missing values
- use different vectorizers/countvectorizers

In [617]:
# Replace all missing values in the DataFrame with 'NA'
X_train.fillna('NA', inplace=True)
X_val.fillna('NA', inplace=True)

In [618]:
print(len(feature_combinations))

31


In [387]:
# Calculate the average length for each text column
text_columns = ['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities']

average_lengths = {}
for column in text_columns:
    # Access the text data in the column
    texts = df[column].astype(str)
    
    # Calculate the length of each text
    lengths = texts.apply(len)
    
    # Compute the average length
    average_length = lengths.mean()
    
    # Store the average length for the column
    average_lengths[column] = average_length

# Create a DataFrame from the average lengths
average_lengths_df = pd.DataFrame.from_dict(average_lengths, orient='index', columns=['Average Length'])

# Print the average lengths for each text column
print(average_lengths_df)

                 Average Length
narrative_tfidf      524.248105
tfidf_title           25.825680
tfidf_keywords       133.045296
events_tfidf         160.490325
entities              14.808471


unique_words(df.narrative_tfidf)  25140  
unique_words(df.tfidf_keywords)   11339  
unique_words(df.events_tfidf)     9315  
unique_words(df.entities)         4617  
unique_words(df.tfidf_title)      2827  

In [619]:
def create_feature_vector(f_set, X_train, X_val, transformations):
    feature_vectors_train = []
    feature_vectors_val = []
    for feature in f_set:
        transformation = transformations[feature]
        transformed_data_train = transformation.fit_transform(X_train[feature])
        transformed_data_val = transformation.transform(X_val[feature])
        
        feature_vectors_train.append(transformed_data_train)
        feature_vectors_val.append(transformed_data_val)
    return hstack(feature_vectors_train), hstack(feature_vectors_val)

In [39]:
# Define your feature sets and their corresponding transformations
transformations = {
    'narrative_tfidf': TfidfVectorizer(ngram_range=(1, 3), max_df=0.95, min_df=2),
    'tfidf_keywords': TfidfVectorizer(ngram_range=(1, 3), max_df=0.95, min_df=2),
    'events_tfidf': TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=2),
    'tfidf_title': CountVectorizer(ngram_range=(1,3), max_df=0.95, min_df=2),
    'entities': CountVectorizer(ngram_range=(1,2), max_df=0.95, min_df=2)
}
algorithms = {
    'Linear SVC': LinearSVC(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier()}

In [622]:
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Initialize an empty list to store individual result DataFrames
result_dfs = []
c=0
# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    c+=1
    X_train_combined, X_val_combined= create_feature_vector(feature_set, X_train, X_val, transformations)
    # Train and evaluate models for each algorithm
    for algorithm_name, algorithm in algorithms.items():
        # Train the model
        algorithm.fit(X_train_combined, y_train)

        # Predict labels
        y_pred = algorithm.predict(X_val_combined)

        # Calculate performance metrics
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred, average='macro')
        recall = recall_score(y_val, y_pred, average='macro')
        f1 = f1_score(y_val, y_pred, average='macro')

        # Create a DataFrame for the current combination and algorithm
        result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                                  'Algorithm': [algorithm_name],
                                  'Accuracy': [accuracy],
                                  'Precision': [precision],
                                  'Recall': [recall],
                                  'F1-Score': [f1]})
        # Append the DataFrame to the list
        result_dfs.append(result_df)
    print('Finished feature combination: {}'.format(c))

Finished feature combination: 1
Finished feature combination: 2
Finished feature combination: 3
Finished feature combination: 4
Finished feature combination: 5
Finished feature combination: 6
Finished feature combination: 7
Finished feature combination: 8
Finished feature combination: 9
Finished feature combination: 10
Finished feature combination: 11
Finished feature combination: 12
Finished feature combination: 13
Finished feature combination: 14
Finished feature combination: 15
Finished feature combination: 16
Finished feature combination: 17
Finished feature combination: 18
Finished feature combination: 19
Finished feature combination: 20
Finished feature combination: 21
Finished feature combination: 22
Finished feature combination: 23
Finished feature combination: 24
Finished feature combination: 25
Finished feature combination: 26
Finished feature combination: 27
Finished feature combination: 28
Finished feature combination: 29
Finished feature combination: 30
Finished feature co

In [623]:
# tem de ser 93
len(result_dfs)

93

In [630]:
# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)

In [17]:
def group_by_alg(results):
    grouped_results = results.groupby('Algorithm')
    algorithm_results = {}

    for algorithm, group in grouped_results:
        algorithm_results[algorithm] = group.copy()

    # Access the results for a specific algorithm
    linear_svc_results = algorithm_results['Linear SVC']
    xgboost_results = algorithm_results['XGBoost']
    random_forest_results = algorithm_results['Random Forest']
    return linear_svc_results, xgboost_results, random_forest_results

In [633]:
linear_svc_results, xgboost_results, random_forest_results= group_by_alg(results)

In [634]:
linear_svc_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_tfidf,Linear SVC,0.657848,0.536161,0.451861,0.463292
3,tfidf_title,Linear SVC,0.545184,0.407933,0.366276,0.369404
6,tfidf_keywords,Linear SVC,0.550832,0.417054,0.367611,0.373840
9,events_tfidf,Linear SVC,0.594828,0.467415,0.395291,0.402478
12,entities,Linear SVC,0.465220,0.316584,0.285509,0.278970
15,"narrative_tfidf, tfidf_title",Linear SVC,0.635553,0.484623,0.451573,0.459932
18,"narrative_tfidf, tfidf_keywords",Linear SVC,0.639120,0.493046,0.439138,0.448661
21,"narrative_tfidf, events_tfidf",Linear SVC,0.648633,0.515646,0.447427,0.458887
24,"narrative_tfidf, entities",Linear SVC,0.628716,0.490489,0.439839,0.450046
27,"tfidf_title, tfidf_keywords",Linear SVC,0.576694,0.443644,0.410214,0.419432


In [635]:
xgboost_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
2,narrative_tfidf,XGBoost,0.645957,0.518850,0.430046,0.435547
5,tfidf_title,XGBoost,0.575505,0.470839,0.372081,0.372104
8,tfidf_keywords,XGBoost,0.567479,0.499054,0.347494,0.339851
11,events_tfidf,XGBoost,0.604637,0.481902,0.387735,0.387493
14,entities,XGBoost,0.490785,0.407892,0.287477,0.267842
17,"narrative_tfidf, tfidf_title",XGBoost,0.656064,0.542607,0.448791,0.460253
20,"narrative_tfidf, tfidf_keywords",XGBoost,0.648930,0.529175,0.431416,0.438787
23,"narrative_tfidf, events_tfidf",XGBoost,0.647146,0.507287,0.432288,0.437332
26,"narrative_tfidf, entities",XGBoost,0.650416,0.523440,0.435376,0.441620
29,"tfidf_title, tfidf_keywords",XGBoost,0.611177,0.497299,0.395066,0.393732


In [637]:
random_forest_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
1,narrative_tfidf,Random Forest,0.647146,0.723007,0.384773,0.361212
4,tfidf_title,Random Forest,0.553508,0.405135,0.356912,0.355992
7,tfidf_keywords,Random Forest,0.587099,0.642391,0.349765,0.331936
10,events_tfidf,Random Forest,0.604935,0.666781,0.357842,0.337512
13,entities,Random Forest,0.477111,0.313748,0.288341,0.278667
16,"narrative_tfidf, tfidf_title",Random Forest,0.652794,0.631934,0.388994,0.366368
19,"narrative_tfidf, tfidf_keywords",Random Forest,0.649524,0.737194,0.386016,0.362386
22,"narrative_tfidf, events_tfidf",Random Forest,0.648930,0.661906,0.386297,0.362770
25,"narrative_tfidf, entities",Random Forest,0.645660,0.722511,0.383781,0.360356
28,"tfidf_title, tfidf_keywords",Random Forest,0.618609,0.586530,0.385004,0.378773


# Embeddings

In [6]:
w2v= KeyedVectors.load_word2vec_format('D:\\MS DATA SCIENCE\\NLP TESE\\embeddings\\skip_s600_word2vec.txt')

In [7]:
w2v

In [8]:
glove= KeyedVectors.load_word2vec_format('D:\\MS DATA SCIENCE\\NLP TESE\\embeddings\\glove_s600.txt')

In [11]:
print('NA' in w2v)
print('NA' in glove)
print('na' in w2v)
print('na' in glove)

False
False
True
True


In [14]:
print('oov' in w2v)
print('oov' in glove)

True
True


In [9]:
y=df['reason']
X = df[['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'events_embeddings', 'entities']]

In [10]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

# Print the shape of each set
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (15701, 5)
y_train shape: (15701,)
X_val shape: (3364, 5)
y_val shape: (3364,)
X_test shape: (3365, 5)
y_test shape: (3365,)


In [11]:
features = ['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'events_embeddings', 'entities']
feature_combinations = []

# Generate the power set of features
for r in range(1, len(features) + 1):
    combinations = itertools.combinations(features, r)
    feature_combinations.extend(combinations)

# Convert each feature combination to a list
feature_combinations = [list(combination) for combination in feature_combinations]
print(len(feature_combinations))

31


In [13]:
w2v['oov']

array([ 2.0180e-02, -5.7960e-03,  2.3383e-02,  4.4911e-02, -4.4600e-04,
        5.5585e-02, -5.2378e-02, -2.6500e-03, -2.6830e-02, -1.4793e-02,
       -1.7029e-02, -1.0867e-02,  1.4438e-02, -6.0080e-03,  4.5800e-03,
        1.8290e-02, -1.9187e-02, -4.6700e-04, -3.8640e-03,  2.4461e-02,
        3.6970e-03,  3.0627e-02,  3.4293e-02,  3.1786e-02, -4.2008e-02,
       -9.0460e-03, -7.7790e-03,  4.3510e-03, -8.2020e-03, -6.9400e-03,
       -2.4276e-02, -5.6580e-03, -4.3717e-02, -4.2580e-03, -1.1344e-02,
       -1.2043e-02,  8.1010e-03, -1.2914e-02,  3.8145e-02, -3.2453e-02,
       -5.0532e-02,  1.7329e-02, -1.8063e-02, -2.7684e-02, -4.2759e-02,
        4.3831e-02,  2.9250e-03, -1.9730e-03,  7.7100e-04, -1.0700e-04,
        2.0503e-02, -3.6539e-02,  4.9820e-03, -1.3998e-02,  1.6050e-03,
        3.9430e-03,  4.8640e-03, -9.0300e-03,  7.8440e-03,  2.6797e-02,
       -4.9060e-03,  1.4060e-03,  1.6700e-02, -2.1390e-03, -1.6850e-03,
       -2.2621e-02,  3.8000e-04,  3.7141e-02,  2.0732e-02, -2.20

In [14]:
glove['oov']

array([ 3.15300e-03,  4.29500e-02, -7.05910e-02,  3.45360e-02,
        6.57250e-02,  1.43640e-01,  6.64080e-02, -1.91811e-01,
       -3.72670e-02,  3.41540e-02, -3.06870e-02, -1.75112e-01,
       -3.16700e-03, -8.82560e-02,  6.90270e-02, -1.71589e-01,
       -1.26465e-01,  3.99500e-02,  1.24014e-01, -8.94050e-02,
       -5.46580e-02,  2.61350e-02,  1.02074e-01, -7.87300e-02,
        2.85975e-01,  4.34770e-02,  1.01158e-01, -1.13990e-01,
       -7.36780e-02, -1.05888e-01,  5.87310e-02, -2.29080e-02,
        3.20710e-02, -3.48900e-03,  5.85940e-02, -2.73537e-01,
       -2.95520e-02, -6.35190e-02,  1.58590e-02,  1.25220e-02,
       -9.80570e-02,  5.83720e-02,  2.83460e-02,  1.42780e-02,
       -3.78080e-02,  5.98400e-02,  7.69130e-02, -4.68600e-03,
       -6.21990e-02, -1.92780e-01,  2.45280e-02,  7.34270e-02,
       -1.28649e-01,  6.83340e-02, -4.64700e-02, -4.69600e-03,
       -9.56590e-02,  8.27820e-02,  2.97810e-02,  3.84990e-02,
       -6.83100e-03,  7.94780e-02,  5.27640e-02,  1.016

### combine all features in the same string  and fillna('')
use feature combinations where at least one feature exists

In [12]:
new_feature_combinations = []

for t in feature_combinations:
    if not df[t].isna().all(axis=1).any():
        new_feature_combinations.append(t)

feature_combinations = new_feature_combinations
print(len(feature_combinations))

28


In [13]:
def embeddings_transformation(X_train_combined,X_val_combined,model):
    # Apply your embedding transformation here
    # initiating a sentence with all zeros
    embedding_size = 600  
    X_train_transformed = np.zeros((len(X_train_combined), embedding_size))
    X_val_transformed = np.zeros((len(X_val_combined), embedding_size))
    
    # Loop over each string in X_train_combined and X_val_combined
    for i, sentence in enumerate(X_train_combined):
        # Loop over each word in the sentence and, if it is in the model's vocabulary, add its feature vector to the total
        embeddings = [model[word] for word in sentence.split() if word in model]
        if embeddings:
            X_train_transformed[i] = np.mean(embeddings, axis=0)

    for i, sentence in enumerate(X_val_combined):
        # Loop over each word in the sentence and, if it is in the model's vocabulary, add its feature vector to the total
        embeddings = [model[word] for word in sentence.split() if word in model]
        if embeddings:
            X_val_transformed[i] = np.mean(embeddings, axis=0)
            
    return X_train_transformed, X_val_transformed

In [14]:
def same_string_combination(algorithms, feature_combinations, X_train, X_val, y_train, y_val, model):
    # Initialize the results table
    cont=0

    # Initialize an empty list to store individual result DataFrames
    result_dfs = []
    # Initialize the results table
    results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Train Acc',
                                    'Precision', 'Recall', 'F1-Score', 'Train F1-Score'])

    # Evaluate models for each feature combination and algorithm
    for feature_set in feature_combinations:
        cont += 1
        # juntar tudo numa string
        X_train_combined = X_train[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
        X_val_combined = X_val[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
        # media dos embeddings
        X_train_transformed, X_val_transformed= embeddings_transformation(X_train_combined,X_val_combined,model)

        # Train and evaluate the models for each algorithm
        for algorithm_name, algorithm in algorithms.items():
            # Fit the algorithm on the training data
            algorithm.fit(X_train_transformed, y_train)

            # Make predictions on the validation data
            y_val_pred = algorithm.predict(X_val_transformed)
            y_train_pred = algorithm.predict(X_train_transformed)

            # Calculate the evaluation metrics
            accuracy = accuracy_score(y_val, y_val_pred)
            train_acc = accuracy_score(y_train, y_train_pred)
            precision = precision_score(y_val, y_val_pred, average='macro')
            recall = recall_score(y_val, y_val_pred, average='macro')
            f1 = f1_score(y_val, y_val_pred, average='macro')
            train_f1 = f1_score(y_train, y_train_pred, average='macro')

            # Create a DataFrame for the current combination and algorithm
            result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                                      'Algorithm': [algorithm_name],
                                      'Accuracy': [accuracy],
                                      'Train Acc': [train_acc],
                                      'Precision': [precision],
                                      'Recall': [recall],
                                      'F1-Score': [f1],
                                      'Train F1-Score': [train_f1]})
            # Append the DataFrame to the list
            result_dfs.append(result_df)    

        print("Tested combination {} of {}".format(cont, len(feature_combinations)))

    # Concatenate all the result DataFrames into a single DataFrame
    results = pd.concat(result_dfs, ignore_index=True)
    return results

* **WORD2VEC**

In [ ]:
algorithms = {'Linear SVC': LinearSVC(), 'XGBoost': XGBClassifier(), 'Random Forest': RandomForestClassifier()}
results= same_string_combination(algorithms, feature_combinations, X_train, X_val, y_train, y_val, w2v)

In [18]:
linear_svc_results, xgboost_results, random_forest_results= group_by_alg(results)

In [19]:
linear_svc_results

,Features Combination,Algorithm,Accuracy,Train Acc,Precision,Recall,F1-Score,Train F1-Score
0,narrative_embeddings,Linear SVC,0.648930,0.662187,0.544382,0.405606,0.393265,0.414176
3,embeddings_title,Linear SVC,0.593044,0.630533,0.477559,0.387300,0.388086,0.443826
6,embeddings_keywords,Linear SVC,0.571046,0.585377,0.351845,0.336720,0.313438,0.341344
9,"narrative_embeddings, embeddings_title",Linear SVC,0.661118,0.671040,0.558042,0.423837,0.417488,0.434297
12,"narrative_embeddings, embeddings_keywords",Linear SVC,0.641795,0.656582,0.467713,0.405228,0.394500,0.418007
15,"narrative_embeddings, events_embeddings",Linear SVC,0.651902,0.661869,0.515856,0.409887,0.398651,0.416957
18,"narrative_embeddings, entities",Linear SVC,0.648633,0.662123,0.501409,0.405071,0.391977,0.414319
21,"embeddings_title, embeddings_keywords",Linear SVC,0.596314,0.623081,0.391366,0.367938,0.354221,0.397412
24,"embeddings_title, events_embeddings",Linear SVC,0.629013,0.654417,0.549317,0.409612,0.409941,0.430290
27,"embeddings_title, entities",Linear SVC,0.590963,0.631425,0.507895,0.384931,0.385991,0.434587


In [20]:
xgboost_results

,Features Combination,Algorithm,Accuracy,Train Acc,Precision,Recall,F1-Score,Train F1-Score
1,narrative_embeddings,XGBoost,0.635256,0.999236,0.509969,0.409663,0.408810,0.999308
4,embeddings_title,XGBoost,0.578181,0.852302,0.454946,0.381049,0.384303,0.816310
7,embeddings_keywords,XGBoost,0.554994,0.998471,0.494531,0.340480,0.332281,0.998463
10,"narrative_embeddings, embeddings_title",XGBoost,0.651902,0.999682,0.533325,0.431033,0.434380,0.999704
13,"narrative_embeddings, embeddings_keywords",XGBoost,0.633472,0.999363,0.509345,0.408900,0.410138,0.999386
16,"narrative_embeddings, events_embeddings",XGBoost,0.640012,0.999299,0.540270,0.416287,0.419548,0.999347
19,"narrative_embeddings, entities",XGBoost,0.644174,0.999363,0.555095,0.421056,0.425631,0.999386
22,"embeddings_title, embeddings_keywords",XGBoost,0.582045,0.999299,0.476507,0.359841,0.350724,0.999411
25,"embeddings_title, events_embeddings",XGBoost,0.613258,0.999172,0.537877,0.391736,0.389994,0.999275
28,"embeddings_title, entities",XGBoost,0.582342,0.945863,0.489473,0.380926,0.387075,0.933150


In [21]:
random_forest_results

,Features Combination,Algorithm,Accuracy,Train Acc,Precision,Recall,F1-Score,Train F1-Score
2,narrative_embeddings,Random Forest,0.622771,0.999490,0.717575,0.372304,0.350126,0.999464
5,embeddings_title,Random Forest,0.574316,0.854149,0.436731,0.368240,0.367079,0.816994
8,embeddings_keywords,Random Forest,0.542509,0.998408,0.696194,0.319561,0.300772,0.998248
11,"narrative_embeddings, embeddings_title",Random Forest,0.626040,0.999809,0.687906,0.375711,0.354295,0.999782
14,"narrative_embeddings, embeddings_keywords",Random Forest,0.626338,0.999490,0.766911,0.371680,0.346759,0.999464
17,"narrative_embeddings, events_embeddings",Random Forest,0.620690,0.999490,0.638213,0.369864,0.346502,0.999465
20,"narrative_embeddings, entities",Random Forest,0.619798,0.999490,0.712264,0.369423,0.345480,0.999465
23,"embeddings_title, embeddings_keywords",Random Forest,0.565993,0.999427,0.704850,0.334087,0.312655,0.999489
26,"embeddings_title, events_embeddings",Random Forest,0.599584,0.999236,0.731725,0.354598,0.330331,0.999314
29,"embeddings_title, entities",Random Forest,0.568371,0.947774,0.473902,0.353109,0.347288,0.934079


* **GLOVE**

In [22]:
algorithms = {'Linear SVC': LinearSVC(max_iter=3000), 'XGBoost': XGBClassifier(), 'Random Forest': RandomForestClassifier()}
results= same_string_combination(algorithms, feature_combinations, X_train, X_val, y_train, y_val, glove)
linear_svc_results, xgboost_results, random_forest_results= group_by_alg(results)

Tested combination 1 of 28
Tested combination 2 of 28
Tested combination 3 of 28
Tested combination 4 of 28
Tested combination 5 of 28
Tested combination 6 of 28
Tested combination 7 of 28
Tested combination 8 of 28
Tested combination 9 of 28
Tested combination 10 of 28
Tested combination 11 of 28
Tested combination 12 of 28
Tested combination 13 of 28
Tested combination 14 of 28
Tested combination 15 of 28
Tested combination 16 of 28
Tested combination 17 of 28
Tested combination 18 of 28
Tested combination 19 of 28
Tested combination 20 of 28
Tested combination 21 of 28
Tested combination 22 of 28
Tested combination 23 of 28
Tested combination 24 of 28
Tested combination 25 of 28
Tested combination 26 of 28
Tested combination 27 of 28
Tested combination 28 of 28


In [23]:
linear_svc_results

,Features Combination,Algorithm,Accuracy,Train Acc,Precision,Recall,F1-Score,Train F1-Score
0,narrative_embeddings,Linear SVC,0.641498,0.663843,0.536399,0.418244,0.416934,0.430110
3,embeddings_title,Linear SVC,0.599287,0.631488,0.549414,0.396139,0.404048,0.450495
6,embeddings_keywords,Linear SVC,0.564209,0.593338,0.416424,0.334178,0.313591,0.347776
9,"narrative_embeddings, embeddings_title",Linear SVC,0.648930,0.674352,0.544240,0.429965,0.431137,0.458724
12,"narrative_embeddings, embeddings_keywords",Linear SVC,0.640309,0.660340,0.529307,0.412868,0.410051,0.435729
15,"narrative_embeddings, events_embeddings",Linear SVC,0.644768,0.664162,0.554611,0.419441,0.418480,0.441116
18,"narrative_embeddings, entities",Linear SVC,0.643876,0.661932,0.558267,0.422543,0.422896,0.428890
21,"embeddings_title, embeddings_keywords",Linear SVC,0.590071,0.623655,0.439151,0.367087,0.357401,0.404446
24,"embeddings_title, events_embeddings",Linear SVC,0.623365,0.656646,0.513260,0.402527,0.399261,0.436912
27,"embeddings_title, entities",Linear SVC,0.587099,0.635119,0.514586,0.385356,0.390115,0.446184


In [24]:
xgboost_results

,Features Combination,Algorithm,Accuracy,Train Acc,Precision,Recall,F1-Score,Train F1-Score
1,narrative_embeddings,XGBoost,0.631688,0.998981,0.526382,0.409532,0.411914,0.999030
4,embeddings_title,XGBoost,0.575803,0.852557,0.458625,0.380369,0.384539,0.815952
7,embeddings_keywords,XGBoost,0.551130,0.997898,0.427491,0.331562,0.316141,0.997931
10,"narrative_embeddings, embeddings_title",XGBoost,0.644174,0.999618,0.547742,0.422298,0.427291,0.999665
13,"narrative_embeddings, embeddings_keywords",XGBoost,0.628121,0.999045,0.493077,0.394957,0.387517,0.999191
16,"narrative_embeddings, events_embeddings",XGBoost,0.626338,0.998917,0.553093,0.409550,0.415101,0.999055
19,"narrative_embeddings, entities",XGBoost,0.626932,0.998917,0.534795,0.404700,0.406896,0.999055
22,"embeddings_title, embeddings_keywords",XGBoost,0.585910,0.999045,0.528772,0.370913,0.369044,0.999255
25,"embeddings_title, events_embeddings",XGBoost,0.614447,0.999045,0.509040,0.388786,0.383101,0.999197
28,"embeddings_title, entities",XGBoost,0.581748,0.945226,0.479469,0.380965,0.386061,0.932707


In [25]:
random_forest_results

,Features Combination,Algorithm,Accuracy,Train Acc,Precision,Recall,F1-Score,Train F1-Score
2,narrative_embeddings,Random Forest,0.612961,0.999490,0.681958,0.366128,0.344859,0.999465
5,embeddings_title,Random Forest,0.569560,0.854213,0.454075,0.365434,0.364975,0.816995
8,embeddings_keywords,Random Forest,0.544887,0.998408,0.696490,0.321050,0.302724,0.998249
11,"narrative_embeddings, embeddings_title",Random Forest,0.620690,0.999809,0.637755,0.368297,0.343312,0.999782
14,"narrative_embeddings, embeddings_keywords",Random Forest,0.613555,0.999490,0.689085,0.364901,0.342812,0.999464
17,"narrative_embeddings, events_embeddings",Random Forest,0.605232,0.999490,0.596097,0.359040,0.336299,0.999464
20,"narrative_embeddings, entities",Random Forest,0.614744,0.999490,0.649214,0.366118,0.344417,0.999464
23,"embeddings_title, embeddings_keywords",Random Forest,0.565398,0.999427,0.698078,0.334187,0.314302,0.999489
26,"embeddings_title, events_embeddings",Random Forest,0.590666,0.999236,0.719322,0.348945,0.326301,0.999314
29,"embeddings_title, entities",Random Forest,0.567479,0.947774,0.469795,0.352270,0.346763,0.934281


### replace missing value with 'NA' (oov) and  mean of each feature+combine
Feature Combination: Once you have encoded the text features using word embeddings, you can combine them in different ways to create new feature representations. 
1. Concatenation: Concatenate the word embeddings of different features together to create a single long vector. For instance, if 'narrative_tfidf' and 'tfidf_title' are both word embeddings of size 300, you can concatenate them to obtain a combined feature vector of size 600.

2. Averaging: Take the average of the word embeddings of different features. This is useful when you want to retain the overall information from each feature but reduce the dimensionality. For example, if 'narrative_tfidf' and 'tfidf_title' are both word embeddings of size 300, you can average them element-wise to obtain a combined feature vector of size 300.

3. Element-wise operations: Perform element-wise operations like addition or multiplication on the word embeddings of different features. This can capture interactions between the features at a more granular level. For example, you can add or multiply the word embeddings of 'tfidf_title' and 'entities' element-wise to obtain a combined feature vector.

Since the texts of each feature are long, if we find the word embeddings of different features and append or concatenate them together, that would result in a longer feature vector. For example, if you we two word embeddings of size 300, concatenating them would result in a combined feature vector of size 600.  

With averagging, the word embeddings of different features are element-wise averaged together. This means that the corresponding elements of the word embeddings are summed up and divided by the number of features. For example, if you we two word embeddings of size 300, averaging them would result in a combined feature vector of size 300. The averaging operation reduces the dimensionality of the feature vector but retains the overall information from each feature.

So since we have 5 features of long texts in separate columns, we are going to perform averaging.  
Calculate the average of the word embeddings for each feature individually. Then, can concatenate the averaged feature vectors to create a single combined feature vector. If each feature_avg has a dimension of 600, and we have 5 features, the dimension of the combined_feature_vector would be 5 * 600 = 3000.

In [12]:
# Convert None values to NaN
# I only get this error for entities and when using embeddings
# Custom function to replace None with 'NA'
def replace_none_with_na(value):
    if value is None:
        return 'NA'
    return value

# Apply the custom function to replace None with 'NA'
X_train = X_train.applymap(replace_none_with_na)
X_val = X_val.applymap(replace_none_with_na)

# Fill NaN values with 'NA'
X_train.fillna('NA', inplace=True)
X_val.fillna('NA', inplace=True)

In [13]:
X_train.entities[9426].split()

['NA']

In [14]:
features = ['narrative_embeddings', 'embeddings_title', 'embeddings_keywords', 'events_embeddings', 'entities']

"Calculate average word embeddings for each column:  
For each column, follow these steps:  

a. Iterate over each row in the column.    
b. Tokenize the text in the current row into individual words.
c. For each word, check if it exists in the word embeddings model. If so, retrieve its word embedding vector.  
d. If any word embeddings are found for the row, calculate the average of all the word embeddings for that row.  
e. Append the average word embedding vector to a list that represents the average word embeddings for the respective column.  

Repeat this process for each column, resulting in 5 lists.  
Combine/concatenate average word embeddings."

In [15]:
# create average word embeddings for a column
def create_emb_column(feature1_text, model):
    feature1_embeddings = np.zeros((len(feature1_text), 600))
    for i, text in enumerate(feature1_text):
        embeddings = [model[word] for word in text.split() if word in model]
        if embeddings:
            feature1_embeddings[i] = np.mean(embeddings, axis=0)
    # axis= 1 mean will be calculated along each row (across columns) of the array, 
    # while axis=0 means that the mean will be calculated along each column (across rows) 
    return feature1_embeddings

In [859]:
tt=create_emb_column(X_train.entities, w2v)

In [860]:
tt.shape

(15701, 600)

In [32]:
# combine the average word embeddings of every column
def combined_feature_vectors(X, feature_set, model):
    embeddings_list=[]
    for feature in feature_set:
        emb= create_emb_column(X[feature], model)
        embeddings_list.append(emb)
    return np.concatenate(embeddings_list, axis=1)

In [33]:
rr=combined_feature_vectors(X_train, ['narrative_embeddings', 'embeddings_title'], w2v)
print(rr.shape)
print(y_train.shape)

(15701, 1200)
(15701,)


In [35]:
algorithms = {'Linear SVC': LinearSVC(max_iter=1100), 'XGBoost': XGBClassifier(), 'Random Forest': RandomForestClassifier()}
# Initialize the results table
cont=0

# Initialize an empty list to store individual result DataFrames
result_dfs = []
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    cont += 1
    X_train_combined = combined_feature_vectors(X_train, feature_set, w2v)
    X_val_combined = combined_feature_vectors(X_val, feature_set, w2v)
    
    # Train and evaluate the models for each algorithm
    for algorithm_name, algorithm in algorithms.items():
        # Fit the algorithm on the training data
        algorithm.fit(X_train_combined, y_train)
        
        # Make predictions on the validation data
        y_val_pred = algorithm.predict(X_val_combined)
        
        # Calculate the evaluation metrics
        accuracy = accuracy_score(y_val, y_val_pred)
        precision = precision_score(y_val, y_val_pred, average='macro')
        recall = recall_score(y_val, y_val_pred, average='macro')
        f1 = f1_score(y_val, y_val_pred, average='macro')
               
        # Create a DataFrame for the current combination and algorithm
        result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                                  'Algorithm': [algorithm_name],
                                  'Accuracy': [accuracy],
                                  'Precision': [precision],
                                  'Recall': [recall],
                                  'F1-Score': [f1]})
        # Append the DataFrame to the list
        result_dfs.append(result_df)    

    print("Tested combination {} of 31".format(cont))

# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)

Tested combination 1 of 31
Tested combination 2 of 31
Tested combination 3 of 31
Tested combination 4 of 31
Tested combination 5 of 31
Tested combination 6 of 31
Tested combination 7 of 31
Tested combination 8 of 31
Tested combination 9 of 31
Tested combination 10 of 31
Tested combination 11 of 31
Tested combination 12 of 31
Tested combination 13 of 31


d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Tested combination 14 of 31
Tested combination 15 of 31
Tested combination 16 of 31
Tested combination 17 of 31
Tested combination 18 of 31
Tested combination 19 of 31
Tested combination 20 of 31
Tested combination 21 of 31
Tested combination 22 of 31
Tested combination 23 of 31
Tested combination 24 of 31
Tested combination 25 of 31
Tested combination 26 of 31
Tested combination 27 of 31
Tested combination 28 of 31
Tested combination 29 of 31
Tested combination 30 of 31
Tested combination 31 of 31


In [36]:
results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_embeddings,Linear SVC,0.648930,0.544382,0.405606,0.393265
1,narrative_embeddings,XGBoost,0.635256,0.509969,0.409663,0.408810
2,narrative_embeddings,Random Forest,0.622771,0.694527,0.370869,0.346914
3,embeddings_title,Linear SVC,0.593044,0.477559,0.387300,0.388086
4,embeddings_title,XGBoost,0.578181,0.454946,0.381049,0.384303
...,...,...,...,...,...,...
88,"embeddings_title, embeddings_keywords, events_...",XGBoost,0.630499,0.546532,0.405986,0.405456
89,"embeddings_title, embeddings_keywords, events_...",Random Forest,0.617717,0.731723,0.367498,0.343267
90,"narrative_embeddings, embeddings_title, embedd...",Linear SVC,0.620690,0.487421,0.452623,0.461120
91,"narrative_embeddings, embeddings_title, embedd...",XGBoost,0.644174,0.514107,0.415637,0.413555


In [37]:
linear_svc_results, xgboost_results, random_forest_results= group_by_alg(results)

In [38]:
linear_svc_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_embeddings,Linear SVC,0.648930,0.544382,0.405606,0.393265
3,embeddings_title,Linear SVC,0.593044,0.477559,0.387300,0.388086
6,embeddings_keywords,Linear SVC,0.571046,0.351845,0.336720,0.313438
9,events_embeddings,Linear SVC,0.597800,0.497920,0.366092,0.351826
12,entities,Linear SVC,0.490785,0.304754,0.284401,0.262690
15,"narrative_embeddings, embeddings_title",Linear SVC,0.653389,0.521191,0.442921,0.447494
18,"narrative_embeddings, embeddings_keywords",Linear SVC,0.641498,0.487156,0.419488,0.418593
21,"narrative_embeddings, events_embeddings",Linear SVC,0.633769,0.500820,0.414289,0.414278
24,"narrative_embeddings, entities",Linear SVC,0.639120,0.478221,0.412406,0.408964
27,"embeddings_title, embeddings_keywords",Linear SVC,0.613555,0.481405,0.411178,0.415653


In [39]:
xgboost_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
1,narrative_embeddings,XGBoost,0.635256,0.509969,0.409663,0.408810
4,embeddings_title,XGBoost,0.578181,0.454946,0.381049,0.384303
7,embeddings_keywords,XGBoost,0.554994,0.494531,0.340480,0.332281
10,events_embeddings,XGBoost,0.593936,0.488321,0.369941,0.362356
13,entities,XGBoost,0.491379,0.352687,0.292087,0.277352
16,"narrative_embeddings, embeddings_title",XGBoost,0.654875,0.585992,0.439103,0.449055
19,"narrative_embeddings, embeddings_keywords",XGBoost,0.640012,0.542839,0.409551,0.408217
22,"narrative_embeddings, events_embeddings",XGBoost,0.641795,0.539568,0.411917,0.410887
25,"narrative_embeddings, entities",XGBoost,0.639120,0.528804,0.414527,0.417030
28,"embeddings_title, embeddings_keywords",XGBoost,0.603151,0.518907,0.396327,0.401726


In [40]:
random_forest_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
2,narrative_embeddings,Random Forest,0.622771,0.694527,0.370869,0.346914
5,embeddings_title,Random Forest,0.572533,0.433916,0.366011,0.364365
8,embeddings_keywords,Random Forest,0.552616,0.701713,0.326997,0.308332
11,events_embeddings,Random Forest,0.584721,0.725815,0.346064,0.326098
14,entities,Random Forest,0.483650,0.336628,0.285540,0.269750
17,"narrative_embeddings, embeddings_title",Random Forest,0.629905,0.710805,0.379238,0.357397
20,"narrative_embeddings, embeddings_keywords",Random Forest,0.622473,0.749885,0.369989,0.346536
23,"narrative_embeddings, events_embeddings",Random Forest,0.618014,0.753809,0.368362,0.346017
26,"narrative_embeddings, entities",Random Forest,0.620690,0.804049,0.369972,0.346374
29,"embeddings_title, embeddings_keywords",Random Forest,0.585910,0.551484,0.357126,0.342500


In [43]:
algorithms = {'Linear SVC': LinearSVC(max_iter=2000), 'XGBoost': XGBClassifier(), 'Random Forest': RandomForestClassifier()}
# Initialize the results table
cont=0

# Initialize an empty list to store individual result DataFrames
result_dfs = []
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    cont += 1
    X_train_combined = combined_feature_vectors(X_train, feature_set, glove)
    X_val_combined = combined_feature_vectors(X_val, feature_set, glove)
    
    # Train and evaluate the models for each algorithm
    for algorithm_name, algorithm in algorithms.items():
        # Fit the algorithm on the training data
        algorithm.fit(X_train_combined, y_train)
        
        # Make predictions on the validation data
        y_val_pred = algorithm.predict(X_val_combined)
        
        # Calculate the evaluation metrics
        accuracy = accuracy_score(y_val, y_val_pred)
        precision = precision_score(y_val, y_val_pred, average='macro')
        recall = recall_score(y_val, y_val_pred, average='macro')
        f1 = f1_score(y_val, y_val_pred, average='macro')
               
        # Create a DataFrame for the current combination and algorithm
        result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                                  'Algorithm': [algorithm_name],
                                  'Accuracy': [accuracy],
                                  'Precision': [precision],
                                  'Recall': [recall],
                                  'F1-Score': [f1]})
        # Append the DataFrame to the list
        result_dfs.append(result_df)    

    print("Tested combination {} of 31".format(cont))

# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)

Tested combination 1 of 31
Tested combination 2 of 31
Tested combination 3 of 31
Tested combination 4 of 31


d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Tested combination 5 of 31


d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Tested combination 6 of 31
Tested combination 7 of 31


...

Tested combination 29 of 31


d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Tested combination 30 of 31


d:\ms data science\nlp tese\envs\env1\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Tested combination 31 of 31


In [44]:
results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_embeddings,Linear SVC,0.641498,0.536399,0.418244,0.416934
1,narrative_embeddings,XGBoost,0.631688,0.526382,0.409532,0.411914
2,narrative_embeddings,Random Forest,0.612663,0.591591,0.363190,0.339340
3,embeddings_title,Linear SVC,0.599287,0.549414,0.396139,0.404048
4,embeddings_title,XGBoost,0.575803,0.458625,0.380369,0.384539
...,...,...,...,...,...,...
88,"embeddings_title, embeddings_keywords, events_...",XGBoost,0.626635,0.570593,0.409803,0.412992
89,"embeddings_title, embeddings_keywords, events_...",Random Forest,0.608502,0.748701,0.361254,0.337296
90,"narrative_embeddings, embeddings_title, embedd...",Linear SVC,0.604935,0.458048,0.443060,0.447426
91,"narrative_embeddings, embeddings_title, embedd...",XGBoost,0.639715,0.523508,0.417121,0.417785


In [45]:
linear_svc_results, xgboost_results, random_forest_results= group_by_alg(results)

In [46]:
linear_svc_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
0,narrative_embeddings,Linear SVC,0.641498,0.536399,0.418244,0.416934
3,embeddings_title,Linear SVC,0.599287,0.549414,0.396139,0.404048
6,embeddings_keywords,Linear SVC,0.564209,0.416424,0.334178,0.313591
9,events_embeddings,Linear SVC,0.601367,0.499280,0.373119,0.363459
12,entities,Linear SVC,0.488704,0.322426,0.285453,0.266673
15,"narrative_embeddings, embeddings_title",Linear SVC,0.650713,0.529241,0.455778,0.465019
18,"narrative_embeddings, embeddings_keywords",Linear SVC,0.637039,0.506177,0.431414,0.436968
21,"narrative_embeddings, events_embeddings",Linear SVC,0.637039,0.504404,0.426846,0.432336
24,"narrative_embeddings, entities",Linear SVC,0.626932,0.488022,0.419109,0.423507
27,"embeddings_title, embeddings_keywords",Linear SVC,0.613853,0.506865,0.425862,0.436067


In [47]:
xgboost_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
1,narrative_embeddings,XGBoost,0.631688,0.526382,0.409532,0.411914
4,embeddings_title,XGBoost,0.575803,0.458625,0.380369,0.384539
7,embeddings_keywords,XGBoost,0.551130,0.427491,0.331562,0.316141
10,events_embeddings,XGBoost,0.589180,0.524140,0.372464,0.370451
13,entities,XGBoost,0.487218,0.326415,0.288997,0.274273
16,"narrative_embeddings, embeddings_title",XGBoost,0.644471,0.556632,0.429398,0.438301
19,"narrative_embeddings, embeddings_keywords",XGBoost,0.625446,0.505681,0.399298,0.397644
22,"narrative_embeddings, events_embeddings",XGBoost,0.631986,0.537057,0.404690,0.404532
25,"narrative_embeddings, entities",XGBoost,0.634067,0.528623,0.410935,0.412303
28,"embeddings_title, embeddings_keywords",XGBoost,0.594530,0.491340,0.387526,0.390440


In [48]:
random_forest_results

,Features Combination,Algorithm,Accuracy,Precision,Recall,F1-Score
2,narrative_embeddings,Random Forest,0.612663,0.591591,0.363190,0.339340
5,embeddings_title,Random Forest,0.569263,0.451494,0.365418,0.366341
8,embeddings_keywords,Random Forest,0.542509,0.696561,0.319561,0.300830
11,events_embeddings,Random Forest,0.576694,0.631101,0.339494,0.318282
14,entities,Random Forest,0.489893,0.324022,0.289234,0.273384
17,"narrative_embeddings, embeddings_title",Random Forest,0.623068,0.587144,0.371616,0.347542
20,"narrative_embeddings, embeddings_keywords",Random Forest,0.619798,0.720878,0.368372,0.344275
23,"narrative_embeddings, events_embeddings",Random Forest,0.603746,0.683864,0.359020,0.337337
26,"narrative_embeddings, entities",Random Forest,0.607610,0.734581,0.362116,0.339916
29,"embeddings_title, embeddings_keywords",Random Forest,0.584126,0.601970,0.357872,0.345924


* svm rbf kernel

In [ ]:
from sklearn.svm import SVC
# Initialize an empty list to store individual result DataFrames
result_dfs = []
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    cont += 1
    X_train_combined = combined_feature_vectors(X_train, feature_set, glove)
    X_val_combined = combined_feature_vectors(X_val, feature_set, glove)
    
    # Create an SVM classifier with the RBF kernel
    algorithm = SVC(kernel='rbf')
    # Fit the algorithm on the training data
    algorithm.fit(X_train_combined, y_train)
    # Make predictions on the validation data
    y_val_pred = algorithm.predict(X_val_combined)
    
    # Calculate the evaluation metrics
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average='macro')
    recall = recall_score(y_val, y_val_pred, average='macro')
    f1 = f1_score(y_val, y_val_pred, average='macro')

    # Create a DataFrame for the current combination and algorithm
    result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                              'Algorithm': [algorithm_name],
                              'Accuracy': [accuracy],
                              'Precision': [precision],
                              'Recall': [recall],
                              'F1-Score': [f1]})
    # Append the DataFrame to the list
    result_dfs.append(result_df)    

    print("Tested combination {} of 31".format(cont))

# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)

In [ ]:
from sklearn.svm import SVC
# Initialize an empty list to store individual result DataFrames
result_dfs = []
# Initialize the results table
results = pd.DataFrame(columns=['Features Combination', 'Algorithm', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Evaluate models for each feature combination and algorithm
for feature_set in feature_combinations:
    cont += 1
    X_train_combined = combined_feature_vectors(X_train, feature_set, w2v)
    X_val_combined = combined_feature_vectors(X_val, feature_set, w2v)
    
    # Create an SVM classifier with the RBF kernel
    algorithm = SVC(kernel='rbf')
    # Fit the algorithm on the training data
    algorithm.fit(X_train_combined, y_train)
    # Make predictions on the validation data
    y_val_pred = algorithm.predict(X_val_combined)
    
    # Calculate the evaluation metrics
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, average='macro')
    recall = recall_score(y_val, y_val_pred, average='macro')
    f1 = f1_score(y_val, y_val_pred, average='macro')

    # Create a DataFrame for the current combination and algorithm
    result_df = pd.DataFrame({'Features Combination': [', '.join(feature_set)],
                              'Algorithm': [algorithm_name],
                              'Accuracy': [accuracy],
                              'Precision': [precision],
                              'Recall': [recall],
                              'F1-Score': [f1]})
    # Append the DataFrame to the list
    result_dfs.append(result_df)    

    print("Tested combination {} of 31".format(cont))

# Concatenate all the result DataFrames into a single DataFrame
results = pd.concat(result_dfs, ignore_index=True)